# Gradiente conjugado

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

## gerando uma matriz simétrica e positiva definida

- Para ser simétrica: $M_{s} = \frac{A + A^{T}}{2}$
- Para ser positiva definida: Todos os autovalores da matriz são positivos
    - Teorema espectral garante que a matriz é diagonalizável (https://pt.wikipedia.org/wiki/Teorema_espectral)
    - Estamos interessados nos autovalores reais, logo a parte complexa no numero deve ser 0
    - Discos de Gershgorin: centramos o valor da diagonal (centro) e somamos os elementos fora da diagonal (raio), em cada linha 
    - Todos os autovalores estão na união desses discos
    - A matriz deve ter a diagonal dominante: a soma dos elementos da diagonal deve ser maior que a soma dos elem fora da diag
    - Com isso podemos estabelecer um teste de rejeição para saber se aquela matriz é positiva definida ou não
    - A matriz não pode ser singular, se for, a solução do sistema não será unica
        -Matriz singular: determinante igual a 0
    

In [ ]:
def DiagDominant(M):
    '''Verifica se a diagonal da matriz é dominante'''
    diag = np.all(2*np.diag(abs(M)))
    soma = np.all(np.sum(abs(M), axis=1))
    if (diag >= soma):   
        return True
    else:
        return False

In [ ]:
def GershgorinDisksTest(M):
    ''' Verifica se todos os autovalores são positivos'''
    n = len(M)
    s = 0
    for i in range(0,n):
        center = M[i][i]
        radius = sum(M[i]) - center
        if(radius - center < 0):
            s+=1
    if(s == 0):
        return True
    else:
        return False

In [ ]:
def Singular(A):
    if(np.linalg.det(A) == 0):
        return True
    else:
        return False

In [ ]:
def generateMatrix(d):
    '''Cria uma matriz simétrica e positiva definida'''
    x = []
    x = [random.randrange(0, d) for i in range(0,d*d)]
    A = np.reshape(x,(d,d))
    if (not DiagDominant(A) and not GershgorinDisksTest(A) and not Singular(A)):
        return generateMatrix(d)
    else: 
        Ms = (A + A.transpose())/2
        return Ms

### Criando o vetor de termos constantes b

In [ ]:
def gererateBvector(n):
    '''Criando o vetor de resultados'''
    b = np.array([random.randrange(1, n) for i in range(0,n)])   
    return b

# Método do Gradiente

http://www.mat.ufrgs.br/~guidi/grad/MAT01169/livro-sci.pdf  
Kim, pag 62 (figura)

- base para o método dos gradientes conjugados
- problema de minimização (matriz positiva e definida garante que existe um mínimo global)
- convergência muito lenta
- minimiza para uma função do tipo $F(x) = \frac{1}{2}x^{T}Ax - b^{T}x + c$
- derivo F(x). $F'(x) = Ax - b$ 
- quer achar o x para que $F(x) = 0$
- a cada iteração pega um x ortogonal ao anterior

# Método dos gradientes conjugados 
Figura Kim, pag 64 

 - O passo de tempo é alterado na direção de menor decrescimento da função
 - é um problema de otimização, a matriz ser positiva definida também garante que podemos encontrar pontos de minimo (det da matriz Hesseana maior que 0)
 - depende apenas do passo imediatamente anterior
 - para um sistema de tamanho N, converge em no máximo N iterações
 - pega direções conjugadas
     - Dizemos que dois vetores diferentes de zero u e v são conjugados (com respeito a $\mathbf {A}$) se $u^{T}Av = 0$ e u e v são não nulos
     - se são ortogonais
     - pega um vetor e corrige naquela direção
 - os resíduos são sempre ortogonais
 

In [ ]:
def GradConjugadoANTIGO(A, b): 
    n = len(b)
    x = np.zeros(n)
    r = b - np.dot(A,x)
    p = r
    rn = r
    k = 0.00001
    for i in range(0,n):
        r = b - np.dot(A,x) 
        alpha = np.dot(r,r)/np.dot(np.dot(p,A),p)
        x = x + np.dot(alpha,p) 
        rn = r - np.dot(alpha,np.dot(A,p))
        if (np.all(np.abs(rn) <= k)):
            print("ooi")
            return x
        beta = np.dot(rn,rn)/np.dot(r,r)
        p = rn - beta*p
    return x

# teste 

In [ ]:
def GradConjugado(A, b):
    x0 = np.zeros(len(b))
    x = x0
    r = b - np.dot(A, x)
    p = r
    rsold = np.dot(np.transpose(r), r)
    for i in range(len(b)):
        Ap = np.dot(A, p)
        alpha = rsold / np.dot(np.transpose(p), Ap)
        x = x + np.dot(alpha, p)
        r = r - np.dot(alpha, Ap)
        rsnew = np.dot(np.transpose(r), r)
        if np.sqrt(rsnew) < 1e-7:
            break
        p = r + (rsnew/rsold)*p #porque nao calcula o beta como o outro método
        rsold = rsnew
    return x

In [ ]:
A = generateMatrix(3)
b = gererateBvector(3)
x = np.linalg.solve(A,b)
ms = GradConjugado(A,b)
si = GradConjugadoANTIGO(A,b)

print("Solução do numpy " + str(x))
print("Minha solução " + str(ms))
print("Solução anterior " + str(si))
print("diferença entre os numpy e minha solução" + str(abs(ms - x))) #praticamente nulo, ordem muito pequena



Solução do numpy [1.  0.  0.5]
Minha solução [1.  0.  0.5]
Solução anterior [0.57552973 0.66165414 0.29460014]
diferença entre os numpy e minha solução[0. 0. 0.]
